 # Phase 1 - Ingestion and Cleaning

 This notebook carries out the following steps
   1. Ingest data downloaded from the LendingClub website
   2. Choose columns to examine and prepare the data set
   3. Visualizes single variable summaries
   4. Removes outliers
   5. Output a dataset ready for later analysis

 Things for you to add
 - Choose 3 to 5 variables and add them to the list of variables below
 - Try visualizing the new variables and other pairs of variables

 Prepare your presentation. Your presentation should contain at most 6 slides.
 1. Begin by giving an overview of the project. What is the problem you wish to solve, what are the objectives? How will you evaluate the performance of the portfolio you provide? How will you measure success? What are the business KPIs?
 2. What variables did you select for further inspection? Why do you think they will be useful? You may support any argument with a visualization.
 3. List any insight you gained by looking at the data visualization or any other data analysis that you perform.
 4. List 3-5 hypothesis about which variables will be important for analysis, and how they will affect the outcome.
 5. State your conclusions. What is the main idea you wish to convey with the presentation? Do you think the data available will be useful to solve the problem?

## Python Code

In [ ]:
import pandas as pd
import os
from sys import platform
import matplotlib.pyplot as plt
import datetime
import numpy as np
import pickle
import seaborn

In [ ]:
def is_integer(x):
    '''
    This function returns True if x is an integer, and False otherwise
    '''
    try:
        return (int(x) == float(x))
    except:
        return False

 ## Parameters

In [ ]:
#You can download the archived data for 2014 from the WayBack machine here: 
# https://web.archive.org/web/20160703081246/https://resources.lendingclub.com/LoanStats3c.csv.zip
# For the 2012-13 data, use https://web.archive.org/web/20160703081246/https://resources.lendingclub.com/LoanStats3b.csv.zip 
# Download both zip files and unzip them.
# Put both these unzipped files in the appropriate folder ("../data" below) before proceeding below.
# The data dictionary is here: https://web.archive.org/web/20200606105339/https://resources.lendingclub.com/LCDataDictionary.xlsx}

In [ ]:
dir_data = "C:\\Users\\rodol\\OneDrive\\Documents\\Training\\Carnegie_Mellon_University\\Courses\\Mini_7\\Business_Value_Analytics\\General\\WK1\\Data"

 ## Step 1 - Ingestion
 Ingest the data files from both sets, perform consistency checks, and prepare one single file for each set

In [ ]:
from typing import Dict, final
def ingest_files(directory: str) -> Dict:
    '''
    This function will ingest every file in the specified directory
    into a pandas dataframe. It will return a dictionary containing
    these dataframes, keyed by the file name.
    
    We assume the directory contains files directly downloaded from
    Lending Club, and *only* those files. Thus, we assume the files are zipped
    (pd.read_csv can read zipped files) and we assume the first line
    in each file needs to be skipped. 
    
    Note that this function will read and ingest more than one file and is
    convenient if you want to ingest data for more than one year at a time.
    
    Note that each file will be read *without* formatting
    '''
    
    # If the directory has no trailing slash, add one
    if directory[-1] != "/":
        directory = directory + "/"
    
    all_files = os.listdir(directory)
    output = {}
    
    print("Directory " + directory + " has " + str(len(all_files)) + " files:")
    for i in all_files:
        print("    Reading file " + i)
        output[i] = pd.read_csv(directory + i, dtype = str, skiprows = 1)
        
        # Some of the files have "summary" lines that, for example
        # read "Total number of loans number in Policy 1: ....."
        # To remove those lines, find any lines with non-integer IDs
        # and remove them
        invalid_rows = (output[i].id.apply( lambda x : is_integer(x) == False ))
        if invalid_rows.sum() > 0:
            print("        Found " + str(invalid_rows.sum()) + " invalid rows which were removed")
            output[i] = output[i][invalid_rows == False]
    
    return output

In [ ]:
# Ingest the set of files we downloaded 
files_data = ingest_files(dir_data)

In [ ]:
files_data.keys()

In [ ]:
data_now = pd.concat(files_data.values()).reset_index(drop = True)
columns = list(data_now.columns)
columns

In [ ]:
data_now.head()

 ## Step 2 - Choose Columns and Prepare Dataset

In [ ]:
# Identify the columns we'll be keeping from the dataset
cols_to_pick = ['id','loan_amnt','funded_amnt','term','int_rate',
                 'installment','grade','emp_length', 'home_ownership',
                 'annual_inc','verification_status','issue_d',
                 'loan_status','purpose','dti', 'delinq_2yrs',
                 'earliest_cr_line','open_acc','pub_rec',
                 'revol_bal','revol_util', 'total_pymnt',
                 'last_pymnt_d', 'recoveries']

# Identify the type of each of these column
float_cols = ['loan_amnt', 'funded_amnt', 'installment', 'annual_inc',
                     'dti', 'revol_bal', 'delinq_2yrs', 'open_acc', 'pub_rec',
                                'total_pymnt', 'recoveries']
cat_cols = ['term', 'grade', 'emp_length', 'home_ownership',
                    'verification_status', 'loan_status', 'purpose']
perc_cols = ['int_rate', 'revol_util']
date_cols = ['issue_d', 'earliest_cr_line', 'last_pymnt_d']

# Ensure that we have types for every column
assert set(cols_to_pick) - set(float_cols) - set(cat_cols) - set(perc_cols) - set(date_cols) == set(["id"])

In [ ]:
# Keep only the columns of interest
final_data = data_now[cols_to_pick].copy()

In [ ]:
final_data.head()

In [ ]:
print("Starting with " + str(len(final_data)) + " rows")

 # To do (A)

 Choose 3 to 5 variables and add them to the list of variables below

 You should consult the data description (excel) file you downloaded to understand the definition of various available columns

 TIP: If you added new variables, be sure to clean them as we just did for the default variables.

 You will have to add them to the group of the right type of variables (e.g. percentage, date, categorical)


In [ ]:
# Update the columns to pick
cols_to_pick.extend(['zip_code', 'sub_grade', 'inq_last_6mths', 'acc_now_delinq', 'mths_since_last_delinq'])

# Update the type of each of these new columns
float_cols.extend(['inq_last_6mths', 'acc_now_delinq', 'mths_since_last_delinq'])
cat_cols.extend(['zip_code', 'sub_grade'])

# Ensure that we have types for every column
assert set(cols_to_pick) - set(float_cols) - set(cat_cols) - set(perc_cols) - set(date_cols) == set(["id"])
# Keep only the columns of interest
final_data = data_now[cols_to_pick].copy()

print("Starting with " + str(len(final_data)) + " rows")

In [ ]:
final_data.head()

 ### Typecast the columns

In [ ]:
for i in float_cols:
    final_data[i] = final_data[i].astype(float)
    
def clean_perc(x):
    if pd.isnull(x):
        return np.nan
    else:
        return float(x.rstrip()[:-1])
    
for i in perc_cols:
    final_data[i] = final_data[i].apply( clean_perc )
    
def clean_date(x):
    if pd.isnull(x):
        return None
    else:
        return datetime.datetime.strptime( x, "%b-%Y").date()
for i in date_cols:
    final_data[i] = final_data[i].apply( clean_date )
    
for i in cat_cols:
    final_data.loc[final_data[i].isnull(), i] = None

In [ ]:
final_data.head()

In [ ]:
final_data.isnull().sum()

In [ ]:
# # Handling missing values
# final_data['emp_length'].fillna('Not Provided', inplace=True)  # Filling missing values with 'Not Provided'
# final_data['revol_util'].fillna(final_data['revol_util'].median(), inplace=True)  # Filling with median
# final_data['last_pymnt_d'].fillna('No Payment', inplace=True)  # Filling missing values with 'No Payment'

# # For 'mths_since_last_delinq', it's a large number of missing values.
# # Depending on business logic, you might want to fill it with a value or create a flag column
# final_data['mths_since_last_delinq'].fillna(999, inplace=True)  # Filling with 999 to signify missing data

# # Check if all missing values are handled
# assert final_data.isnull().sum().sum() == 0

 ## Step 3- Visualize the variables

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(4,4))
def visualize_columns():
    
    '''
    This function visualizes all columns
      - Box-and-whisker plots for continuous variables
      - Lists of distinct values for categorical columns
      - A timeline density for dates
    '''
    
    # Float columns
    for i in float_cols + perc_cols:
        # seaborn.boxplot(final_data[i])
        final_data.boxplot(i)

        # Print the three highest values
        highest_vals = sorted(final_data[i], reverse=True)[:3]
        smallest_val = min(final_data[i])
        print("Top 3 Max: ", highest_vals, "\nMin: ", smallest_val)
        # plt.text(smallest_val, -0.3, highest_vals[0])
        # plt.text(smallest_val, -0.2, highest_vals[1])
        # plt.text(smallest_val, -0.1, highest_vals[2])

        plt.show()
        
    # Categorical columns 
    for i in cat_cols:
        print(i)
        print(str(len(set(final_data[i]))) + " distinct values")
        print(final_data[i].value_counts())
        print("")
        print("")
    
    # Date columns
    for i in date_cols:
        final_data[final_data[i].isnull() == False][i].apply(lambda x : str(x.year) +
                                                "-" + str(x.month)).value_counts(ascending = True).plot()
        plt.title(i + " (" + str(final_data[i].isnull().sum()) + " null values)")
        plt.show()

In [ ]:
visualize_columns()

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# def visualize_columns(df, float_cols, perc_cols, cat_cols, date_cols):
#     '''
#     This function visualizes all columns
#       - Box-and-whisker plots for continuous variables
#       - Lists of distinct values for categorical columns
#       - A timeline density for dates
#     '''
    
#     # Float columns
#     for i in float_cols + perc_cols:
#         plt.figure(figsize=(8, 4))
#         sns.boxplot(x=df[i])
#         plt.grid(True)
        
#         # Print the three highest values
#         highest_vals = sorted(df[i].dropna(), reverse=True)[:3]
#         smallest_val = min(df[i].dropna())
#         plt.title(f"{i}\nTop 3 Max: {highest_vals} \nMin: {smallest_val}")
#         plt.show()
        
#     # Categorical columns 
#     for i in cat_cols:
#         plt.figure(figsize=(8, 4))
#         sns.countplot(data=df, x=i, order=df[i].value_counts().index)
#         plt.title(f"{i} - {len(df[i].unique())} distinct values")
#         plt.xticks(rotation=90)
#         plt.grid(True)
#         plt.show()
    
#     # Date columns
#     for i in date_cols:
#         plt.figure(figsize=(8, 4))
#         df[df[i].isnull() == False][i].apply(lambda x : str(x.year) +
#                                             "-" + str(x.month)).value_counts(ascending = True).sort_index().plot(kind='bar')
#         plt.title(f"{i} ({df[i].isnull().sum()} null values)")
#         plt.grid(True)
#         plt.show()

# # Usage example:
# # visualize_columns(final_data, float_cols, perc_cols, cat_cols, date_cols)

 # To do (B)

 Try visualizing the new variables and other pairs of variables


In [ ]:
import plotly.express as px

def visualize_new_columns_with_plotly(final_data, float_cols, cat_cols):
    
    # Float columns
    for i in float_cols:
        fig = px.box(final_data, y=i, title=f"Boxplot of {i}")
        fig.show()
        
        # Print the three highest values and the smallest value
        highest_vals = sorted(final_data[i].dropna(), reverse=True)[:3]
        smallest_val = final_data[i].min()
        print(f"Top 3 Max for {i}: {highest_vals} \nMin for {i}: {smallest_val}\n")
        
    # Categorical columns
    for i in cat_cols:
        if i == 'zip_code':
            # Group by the first 3 digits of the zip code to represent broader geographical regions
            final_data['zip_region'] = final_data['zip_code'].str[:3]
            fig = px.histogram(final_data, x='zip_region', title=f"Countplot of {i} by region")
        else:
            fig = px.histogram(final_data, x=i, title=f"Countplot of {i}")
        
        fig.show()
        
        print(f"{i} has {len(final_data[i].unique())} distinct values.")
        print(final_data[i].value_counts(), '\n')

# Newly added float columns
new_float_cols = ['inq_last_6mths', 'acc_now_delinq', 'mths_since_last_delinq']

# Newly added categorical columns
new_cat_cols = ['zip_code', 'sub_grade']

# Assuming final_data contains your actual data
# final_data = data_now[cols_to_pick].copy()

# Call the visualize_new_columns_with_plotly function
visualize_new_columns_with_plotly(final_data, new_float_cols, new_cat_cols)

In [ ]:
# # Your existing lists of columns and their types
# cols_to_pick = ['id','loan_amnt','funded_amnt','term','int_rate',
#                 'installment','grade','emp_length','home_ownership',
#                 'annual_inc','verification_status','issue_d',
#                 'loan_status','purpose','dti','delinq_2yrs',
#                 'earliest_cr_line','open_acc','pub_rec',
#                 'revol_bal','revol_util','total_pymnt',
#                 'last_pymnt_d','recoveries', 'zip_code', 'sub_grade', 
#                 'inq_last_6mths', 'acc_now_delinq', 'mths_since_last_delinq']

# float_cols = ['loan_amnt', 'funded_amnt', 'installment', 'annual_inc',
#               'dti', 'revol_bal', 'delinq_2yrs', 'open_acc', 'pub_rec',
#               'total_pymnt', 'recoveries', 'inq_last_6mths', 'acc_now_delinq', 
#               'mths_since_last_delinq']

# cat_cols = ['term', 'grade', 'emp_length', 'home_ownership',
#             'verification_status', 'loan_status', 'purpose', 'zip_code', 'sub_grade']

# perc_cols = ['int_rate', 'revol_util']

# date_cols = ['issue_d', 'earliest_cr_line', 'last_pymnt_d']

# # Ensure that we have types for every column
# assert set(cols_to_pick) - set(float_cols) - set(cat_cols) - set(perc_cols) - set(date_cols) == set(["id"])

# # Assuming final_data contains your actual data
# # final_data = data_now[cols_to_pick].copy()

# # Call the visualize_columns function
# visualize_columns(final_data, float_cols, perc_cols, cat_cols, date_cols)

 ## Step 4 - Handle outliers

In [ ]:
# There are quite a few outliers, but the two most obvious
# ones to remove are in annual_inc, revol_bal Remove these.
n_rows = len(final_data)
final_data = final_data[final_data.annual_inc < 1000000]
final_data = final_data[final_data.revol_bal < 400000]
final_data = final_data[final_data.dti < 200]
print("Removed " + str(n_rows - len(final_data)) + " rows")

In [ ]:
# Remove all loans that are too recent to have been paid off or
# defaulted
n_rows = len(final_data)
final_data = final_data[final_data.loan_status.isin(['Fully Paid','Charged Off','Default'])]
print("Removed " + str(n_rows - len(final_data)) + " rows")

In [ ]:
# Only include loans issued since 2009
n_rows = len(final_data)
final_data = final_data[final_data.issue_d >= datetime.date(2009, 1, 1)]
print("Removed " + str(n_rows - len(final_data)) + " rows")

In [ ]:
# Visualize the data again
visualize_columns()

 ### Drop null values

In [ ]:
# Deal with null values. We allow categorical variables to be null
# OTHER than grade, which is a particularly important categorical.
# All non-categorical variables must be non-null, and we drop
# rows that do not meet this requirement
required_cols = set(cols_to_pick) - set(cat_cols) - set(["id"])
required_cols.add("grade")

n_rows = len(final_data)
final_data.dropna(subset = required_cols ,inplace=True)
print("Removed " + str(n_rows - len(final_data)) + " rows")

In [ ]:
# Some of the columns selected will not be used directly in the model, but will be used to generate other features.
# Create variables specifying the features that will be used

# All categorical columns other than "loan_status" will be used as discrete features
discrete_features = list(set(cat_cols) - set(["loan_status"]))

# All numeric columns will be used as continuous features
continuous_features = list(float_cols + perc_cols)

 ## Step 5 - Save a Pickle

In [ ]:
# Remove the total_pymnt from the list of continuous features; this
# variable is highly predictive of the outcome but is not known at
# the time the loan is issued
continuous_features = [i for i in continuous_features if i not in ["total_pymnt", "recoveries"]]

In [ ]:
# Define the output path for the pickle
pickle_file = "/".join(['.', "PickleData", "clean_data.pickle"])
os.makedirs(os.path.dirname(pickle_file), exist_ok=True)
pickle.dump( [final_data, discrete_features, continuous_features], open(pickle_file, "wb") )

In [ ]:
final_data.head()

 ## Read from Pickle if Saved
 Read data from saved pickle files

In [ ]:
### Read the data and features from the pickle
final_data, discrete_features, continuous_features = pickle.load( open( "./PickleData/clean_data.pickle", "rb" ) )
#final_data, discrete_features, continuous_features = pickle.load( open( "./PickleData/201213clean_data.pickle", "rb" ) )

In [ ]:
final_data.head()

 ## Step 6 Prepare your presentation.

 Your presentation should contain at most 6 slides.

 1) Begin by giving an overview of the project. What is the problem you wish to solve, what are the objectives?

 How will you evaluate the performance of the portfolio you provide? How will you measure success?

 What are the business KPIs?

 2) What variables did you select for further inspection? Why do you think they will be useful?

 You may support any argument with a visualization.

 3) List any insight you gained by looking at the data visualization or any other data analysis that you perform.

 4) List 3-5 hypothesis about which variables will be important for analysis, and how they will affect the outcome.

 5) State your conclusions. What is the main idea you wish to convey with the presentation? Do you think the data available will be useful to solve the problem?
